In [1]:
"""Plot MED OFF vs. ON power (Figures 2 & S2)."""

import os
import sys
from pathlib import Path
import numpy as np
import copy
import pandas as pd
import pte_stats
import pte_decode
from matplotlib import pyplot as plt
from scipy.stats import sem
from matplotlib import pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

cd_path = Path(os.getcwd()).absolute().parent
sys.path.append(os.path.join(cd_path, "coherence"))

import matplotlib
matplotlib.rc('xtick', labelsize=6)
matplotlib.rc('ytick', labelsize=6)
matplotlib.rc('legend', fontsize=6)
matplotlib.rc("font", size=6, family="Arial")
matplotlib.rc('axes', labelsize=7)
matplotlib.rc('axes', titlesize=7)
matplotlib.rcParams["pdf.fonttype"] = 42
matplotlib.rcParams["ps.fonttype"] = 42

prop_cycle = plt.rcParams["axes.prop_cycle"]

n_perm = 100_000
alpha = 0.05
two_sided = True

FOLDERPATH_ANALYSIS = "Path_to\\Project\\Analysis"
FOLDERPATH_FIGURES = os.path.join(os.path.dirname(os.getcwd()), "figures")

lfreq = 4.0
hfreq = 40.0

fbands = {
    "alpha": (8, 12),
    "low_beta": (12, 20),
    "high_beta": (20, 30)
}

fband_func = np.mean

In [ ]:
# WHOLE CORTEX POWER (Figure 2a; also Figure S2 when adjusting the data loaded)
data = pd.read_pickle(os.path.join(
    FOLDERPATH_ANALYSIS, "task-Rest_acq-multi_run-multi_pow_standard_whole-MedOffOn_multi_sub.pkl")
)
freqs = copy.deepcopy(data["freqs"][0])
freqs_delta = np.unique(np.subtract(freqs[1:], freqs[:-1]))[0]
data["freqs"] = [freqs] * len(data["ch_types"])
data = pd.DataFrame.from_dict(data)

pow_off = np.array(data["power-multitaper"][(data["med"] == "Off") & (data["ch_regions"] == "cortex")].to_list())
pow_on = np.array(data["power-multitaper"][(data["med"] == "On") & (data["ch_regions"] == "cortex")].to_list())

np.random.seed(44)

lfreq_i = freqs.index(lfreq)
hfreq_i = freqs.index(hfreq)
plot_freqs = freqs[lfreq_i:hfreq_i+1]

pow_off = pow_off[:, lfreq_i:hfreq_i+1]
pow_on = pow_on[:, lfreq_i:hfreq_i+1]

fig, axis = pte_decode.lineplot_prediction_compare(
    x_1=pow_off.T,
    x_2=pow_on.T,
    times=np.arange(lfreq, hfreq+0.5, 0.5),
    data_labels=["OFF therapy", "ON levodopa"],
    x_label="Frequency (Hz)",
    y_label="Power (% total)",
    two_tailed=True,
    paired_x1x2=True,
    n_perm=n_perm,
    correction_method="cluster_pvals",
    title="Cortex",
    colour=["#DF4A4A", "#376EB4"]
)

for fband_name, fband_lims in fbands.items():
    stat, pval = pte_stats.permutation_onesample(
        data_a=fband_func(pow_off[:, freqs.index(fband_lims[0]):freqs.index(fband_lims[1])+1], 1),
        data_b=fband_func(pow_on[:, freqs.index(fband_lims[0]):freqs.index(fband_lims[1])+1], 1),
        n_perm=n_perm, two_tailed=two_sided
    )
    if pval < alpha:
        axis.axvspan(fband_lims[0], fband_lims[1], color=[0.8, 0.8, 0.8], alpha=0.3, linewidth=0, zorder=0)
        prefix = "[SIGNIFICANT]"
    else:
        prefix = ""
    print(f"{prefix} {fband_name}: stat={stat}; p-val={pval}")

axis.spines['top'].set_visible(False)
axis.spines['right'].set_visible(False)
axis.set_box_aspect(0.8)
axis.legend(handlelength=0.5, handletextpad=0.5, labelspacing=0, borderaxespad=0, loc="lower left")
leg = axis.get_legend()
leg.legend_handles[0].set_linewidth(1.5)
leg.legend_handles[1].set_linewidth(1.5)
leg.legend_handles[2].set_color([0.5, 0.5, 0.5])
leg.set_frame_on(False)
axis.set_xticks(np.arange(5, 45, 5))
axis.tick_params("x", pad=2, size=2)
axis.tick_params("y", pad=2, size=2)
axis.xaxis.labelpad = 0
axis.yaxis.labelpad = 1

fig.set_size_inches(1.6, 1.6)

fig.savefig(os.path.join(FOLDERPATH_FIGURES, "Manuscript_Power_Med_whole_cortex.pdf"), dpi=300)

In [ ]:
# STN POWER (Figure 2a; also Figure S2 when adjusting the data loaded)
data = pd.read_pickle(os.path.join(
    FOLDERPATH_ANALYSIS, "task-Rest_acq-multi_run-multi_pow_standard_whole-MedOffOn_multi_sub.pkl")
)
freqs = copy.deepcopy(data["freqs"][0])
freqs_delta = np.unique(np.subtract(freqs[1:], freqs[:-1]))[0]
data["freqs"] = [freqs] * len(data["ch_types"])
data = pd.DataFrame.from_dict(data)

pow_off = np.array(data["power-multitaper"][(data["med"] == "Off") & (data["ch_regions"] == "STN")].to_list())
pow_on = np.array(data["power-multitaper"][(data["med"] == "On") & (data["ch_regions"] == "STN")].to_list())

np.random.seed(44)

lfreq_i = freqs.index(lfreq)
hfreq_i = freqs.index(hfreq)
plot_freqs = freqs[lfreq_i:hfreq_i+1]

pow_off = pow_off[:, lfreq_i:hfreq_i+1]
pow_on = pow_on[:, lfreq_i:hfreq_i+1]

fig, axis = pte_decode.lineplot_prediction_compare(
    x_1=pow_off.T,
    x_2=pow_on.T,
    times=np.arange(lfreq, hfreq+0.5, 0.5),
    data_labels=["OFF therapy", "ON levodopa"],
    x_label="Frequency (Hz)",
    y_label="Power (% total)",
    two_tailed=True,
    paired_x1x2=True,
    n_perm=n_perm,
    correction_method="cluster_pvals",
    title="STN",
    colour=["#DF4A4A", "#376EB4"]
)

for fband_name, fband_lims in fbands.items():
    stat, pval = pte_stats.permutation_onesample(
        data_a=fband_func(pow_off[:, freqs.index(fband_lims[0]):freqs.index(fband_lims[1])+1], 1),
        data_b=fband_func(pow_on[:, freqs.index(fband_lims[0]):freqs.index(fband_lims[1])+1], 1),
        n_perm=n_perm, two_tailed=two_sided
    )
    if pval < alpha:
        axis.axvspan(fband_lims[0], fband_lims[1], color=[0.8, 0.8, 0.8], alpha=0.3, linewidth=0)
        prefix = "[SIGNIFICANT]"
    else:
        prefix = ""
    print(f"{prefix} {fband_name}: stat={stat}; p-val={pval}")

axis.set_ylim((0.2, 3.1))

axis.spines['top'].set_visible(False)
axis.spines['right'].set_visible(False)
axis.set_box_aspect(0.8)
axis.legend(handlelength=0.5, handletextpad=0.5, labelspacing=0, borderaxespad=0, loc="lower left")
leg = axis.get_legend()
leg.legend_handles[0].set_linewidth(1.5)
leg.legend_handles[1].set_linewidth(1.5)
leg.legend_handles[2].set_color([0.5, 0.5, 0.5])
leg.set_frame_on(False)
axis.set_xticks(np.arange(5, 45, 5))
axis.tick_params("x", pad=2, size=2)
axis.tick_params("y", pad=2, size=2)
axis.xaxis.labelpad = 0
axis.yaxis.labelpad = 1
axis.set_yticks([1, 2, 3])

fig.set_size_inches(1.58, 1.58)

fig.savefig(os.path.join(FOLDERPATH_FIGURES, "Manuscript_Power_Med_STN.pdf"), dpi=300)

In [ ]:
# STN POWER for EL008 (Figure 2c inset)
FOLDERPATH_ANALYSIS_EL008 = "Path_to\\Project\\Analysis\\Results\\BIDS_01_Berlin_Neurophys\\sub-EL008\\ses-multi"

data = pd.read_pickle(
    os.path.join(
        FOLDERPATH_ANALYSIS_EL008,
        "task-Rest_acq-multi_run-1_pow_standard_channels-MedOffOn_single_sub.pkl",
    )
)
freqs = copy.deepcopy(data["freqs"][0])
freqs_delta = np.unique(np.subtract(freqs[1:], freqs[:-1]))[0]
data["freqs"] = [freqs] * len(data["ch_types"])
data = pd.DataFrame.from_dict(data)

off_idcs = data.index[(data["med"] == "Off") & (data["ch_regions"] == "STN")]

ch_names = data["ch_names"][off_idcs]

ch_nums = [str(num) for num in range(1, 17)]
ch_nums.pop(ch_nums.index("13"))
for num_idx, num in enumerate(ch_nums):
    if len(num) == 1:
        ch_nums[num_idx] = f"0{num}"

ch_data = {num: [] for num in ch_nums}
for key in ch_data.keys():
    for name_idx, name in enumerate(ch_names):
        if key in name:
            ch_data[key].append(data["power-multitaper"][off_idcs[name_idx]])
    ch_data[key] = np.mean(ch_data[key], axis=0)

pow_off = np.array(list(ch_data.values()))


patterns = pd.DataFrame.from_dict(
    pd.read_pickle(
        os.path.join(
            FOLDERPATH_ANALYSIS,
            "task-Rest_acq-multi_run-multi_pow_ssd_topography_raw-MedOffOn_multi_sub.pkl",
        )
    )
)
off_idcs = patterns.index[
    (patterns["med"] == "Off")
    & (patterns["ch_regions"] == "STN")
    & (patterns["band_names"] == "low_beta")
    & (patterns["component_numbers"] == 1)
    & (patterns["sub"] == "008")
]
ch_names = [
    "01", "02", "03",
    "04", "05", "06",
    "07", "08", "09",
    "10", "11", "12",
          "14", "15",
          "16",
]
ch_data = []
ch_data_med = []
patterns = patterns.loc[off_idcs]
for ch_name in ch_names:
    ch_data.append([])
    for row_i in patterns.index:
        if ch_name in patterns["ch_names"].loc[row_i]:
            ch_data[-1].append(patterns["ssd_topographies"].loc[row_i])
for val_i, val in enumerate(ch_data):
    ch_data[val_i] = np.mean(val)

lfreq_i = freqs.index(9)
hfreq_i = freqs.index(31)
plot_freqs = freqs[lfreq_i : hfreq_i + 1]

pow_off = pow_off[:, lfreq_i : hfreq_i + 1]

fig, axis = plt.subplots(1, 1)

prop_cycle = plt.rcParams["axes.prop_cycle"]
def_colors = prop_cycle.by_key()["color"]

n_colours = 256
colours = plt.get_cmap("Reds")(range(n_colours))
cmap = LinearSegmentedColormap.from_list(name="red_alpha", colors=colours)

cmap = plt.get_cmap("viridis")

normalise = matplotlib.colors.Normalize(
    vmin=np.min(ch_data), vmax=np.max(ch_data)
)
colours = cmap(normalise(ch_data))

for ch_idx, colour in enumerate(colours):
    if ch_idx not in [6, 7, 8]:
        axis.plot(plot_freqs, pow_off[ch_idx], color=colour, linewidth=1)

for ch_idx, colour in enumerate(colours):
    if ch_idx in [6, 7, 8]:
        axis.plot(plot_freqs, pow_off[ch_idx], color=colour, linewidth=1)

axis.set_ylim(0.5, 2.5)
axis.set_xlabel("Frequency (Hz)")
axis.set_ylabel("Power (% total)")
axis.spines["top"].set_visible(False)
axis.spines["right"].set_visible(False)
axis.set_xticks(np.arange(10, 31, 10))
axis.set_yticks([0.5, 2.5])
axis.tick_params("x", pad=2, size=2)
axis.tick_params("y", pad=2, size=2)
axis.xaxis.labelpad = 0
axis.yaxis.labelpad = -8
axis.set_title("Subject EL008")

fig.set_size_inches(1.1, 1.1)

fig.savefig(
    os.path.join(FOLDERPATH_FIGURES, "Manuscript_Power_EL008_MedOFF_STN.pdf"), dpi=300
)

In [ ]:
# WHOLE CORTEX SSD POWER (Figure 2b inset)
data = pd.read_pickle(os.path.join(
    FOLDERPATH_ANALYSIS, "task-Rest_acq-multi_run-multi_pow_ssd_noavg-MedOffOn_multi_sub.pkl")
)
freqs = copy.deepcopy(data["frequencies"][0])
del data["frequencies"]
data = pd.DataFrame.from_dict(data)

fband_label = "high_beta"
lfreq = 19
hfreq = 31

power_off = np.array(data["power-ssd"][(data["med"] == "Off") & (data["transformed_ch_regions"] == "cortex")
                        & (data["band_names"] == fband_label) & (data["component_numbers"] == 1)].tolist())
power_on = np.array(data["power-ssd"][(data["med"] == "On") & (data["transformed_ch_regions"] == "cortex")
                        & (data["band_names"] == fband_label) & (data["component_numbers"] == 1)].tolist())

lfreq_i = freqs.index(lfreq)
hfreq_i = freqs.index(hfreq)
plot_freqs = freqs[lfreq_i:hfreq_i+1]
power_off = power_off[:, lfreq_i:hfreq_i+1]
power_on = power_on[:, lfreq_i:hfreq_i+1]

power_off_sem = sem(power_off, axis=0)
power_on_sem = sem(power_on, axis=0)

fig, axis = plt.subplots(1, 1)

axis.plot(plot_freqs, power_off.mean(0), color="#DF4A4A", label="OFF therapy")
axis.fill_between(plot_freqs, power_off.mean(0)-power_off_sem, power_off.mean(0)+power_off_sem, color="#DF4A4A", alpha=0.3)
axis.plot(plot_freqs, power_on.mean(0), color="#376EB4", label="ON levodopa")
axis.fill_between(plot_freqs, power_on.mean(0)-power_on_sem, power_on.mean(0)+power_on_sem, color="#376EB4", alpha=0.3)


axis.spines['top'].set_visible(False)
axis.spines['right'].set_visible(False)
axis.set_xticks(np.arange(20, 31, 5))
axis.set_yticks([0, 2])
axis.set_ylim(0, 2)
axis.tick_params("x", pad=2, size=2)
axis.tick_params("y", pad=2, size=2)
axis.set_xlabel("Frequency (Hz)")
axis.set_ylabel("Power (dB)")
fig.suptitle("Cortex")
axis.xaxis.labelpad = 0
axis.yaxis.labelpad = -5

fig.set_size_inches(1, 1)

fig.savefig(os.path.join(FOLDERPATH_FIGURES, "Manuscript_Power_SSD_spectrum_Med_cortex.pdf"), dpi=300)

In [ ]:
# WHOLE STN SSD POWER (Figure 2b inset)
data = pd.read_pickle(os.path.join(
    FOLDERPATH_ANALYSIS, "task-Rest_acq-multi_run-multi_pow_ssd_noavg-MedOffOn_multi_sub.pkl")
)
freqs = copy.deepcopy(data["frequencies"][0])
del data["frequencies"]
data = pd.DataFrame.from_dict(data)

fband_label = "low_beta"
lfreq = 10
hfreq = 22

power_off = np.array(data["power-ssd"][(data["med"] == "Off") & (data["transformed_ch_regions"] == "STN")
                        & (data["band_names"] == fband_label) & (data["component_numbers"] == 1)].tolist())
power_on = np.array(data["power-ssd"][(data["med"] == "On") & (data["transformed_ch_regions"] == "STN")
                        & (data["band_names"] == fband_label) & (data["component_numbers"] == 1)].tolist())

np.random.seed(44)

lfreq_i = freqs.index(lfreq)
hfreq_i = freqs.index(hfreq)
plot_freqs = freqs[lfreq_i:hfreq_i+1]
power_off = power_off[:, lfreq_i:hfreq_i+1]
power_on = power_on[:, lfreq_i:hfreq_i+1]

power_off_sem = sem(power_off, axis=0)
power_on_sem = sem(power_on, axis=0)

fig, axis = plt.subplots(1, 1)

axis.plot(plot_freqs, power_off.mean(0), color="#DF4A4A", label="OFF therapy")
axis.fill_between(plot_freqs, power_off.mean(0)-power_off_sem, power_off.mean(0)+power_off_sem, color="#DF4A4A", alpha=0.3)
axis.plot(plot_freqs, power_on.mean(0), color="#376EB4", label="ON levodopa")
axis.fill_between(plot_freqs, power_on.mean(0)-power_on_sem, power_on.mean(0)+power_on_sem, color="#376EB4", alpha=0.3)

axis.spines['top'].set_visible(False)
axis.spines['right'].set_visible(False)
axis.set_xticks(np.arange(12, 21, 4))
axis.set_yticks([0, 2])
axis.set_ylim(0, 2)
axis.tick_params("x", pad=2, size=2)
axis.tick_params("y", pad=2, size=2)
axis.set_xlabel("Frequency (Hz)")
axis.set_ylabel("Power (dB)")
fig.suptitle("STN")
axis.xaxis.labelpad = 0
axis.yaxis.labelpad = -5

fig.set_size_inches(1, 1)

fig.savefig(os.path.join(FOLDERPATH_FIGURES, "Manuscript_Power_SSD_spectrum_Med_STN.pdf"), dpi=300)